# 🤖 Generador de Datasets con Modelos de Lenguaje

Este notebook te permite generar datasets utilizando diversos modelos:
- **Modelos de Hugging Face** (gratuitos, ejecutados localmente o via Inference API)
- **Modelos Frontera** (GPT-4, Claude, Gemini via APIs)

## 📋 Índice
1. Instalación de dependencias
2. Configuración de APIs
3. Modelos de Hugging Face
4. Modelos Frontera (APIs)
5. Generación de Datasets
6. Exportación de Resultados

## 1. Instalación de Dependencias

In [ ]:
!pip install -q transformers accelerate bitsandbytes
!pip install -q openai anthropic google-generativeai
!pip install -q datasets pandas tqdm

In [ ]:
import os
import json
import pandas as pd
from tqdm.auto import tqdm
from typing import List, Dict, Optional
import time

## 2. Configuración de APIs

Configura tus claves API aquí. Puedes usar Google Colab Secrets para mayor seguridad:
- Ve a (icono de llave) en el panel izquierdo
- Añade tus secrets con los nombres correspondientes

In [ ]:
# Opción 1: Usando Google Colab Secrets (recomendado)
try:
    from google.colab import userdata
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    ANTHROPIC_API_KEY = userdata.get('ANTHROPIC_API_KEY')
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    HF_TOKEN = userdata.get('HF_TOKEN')
except:
    # Opción 2: Configuración directa (menos seguro)
    OPENAI_API_KEY = ""  # tu-api-key-aqui
    ANTHROPIC_API_KEY = ""  # tu-api-key-aqui
    GOOGLE_API_KEY = ""  # tu-api-key-aqui
    HF_TOKEN = ""  # tu-huggingface-token-aqui

## 3. Configuración de Modelos de Hugging Face

Aquí configuramos modelos open-source que se pueden ejecutar en Colab (con limitaciones de GPU)

In [ ]:
class HuggingFaceGenerator:
    """Generador usando modelos de Hugging Face"""

    def __init__(self, model_name: str = "meta-llama/Llama-3.2-3B-Instruct", use_4bit: bool = True):
        """
        Modelos recomendados para Colab (gratuito):
        - meta-llama/Llama-3.2-3B-Instruct (3B parámetros)
        - mistralai/Mistral-7B-Instruct-v0.3 (7B parámetros, requiere más GPU)
        - google/gemma-2-2b-it (2B parámetros, muy ligero)
        - Qwen/Qwen2.5-3B-Instruct (3B parámetros)
        """
        from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
        import torch

        self.model_name = model_name
        print(f"Cargando modelo: {model_name}")

        # Configuración para reducir uso de memoria
        if use_4bit:
            quantization_config = BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_compute_dtype=torch.float16
            )
        else:
            quantization_config = None

        self.tokenizer = AutoTokenizer.from_pretrained(model_name, token=HF_TOKEN)
        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            quantization_config=quantization_config,
            device_map="auto",
            token=HF_TOKEN
        )
        print("Modelo cargado exitosamente")

    def generate(self, prompt: str, max_tokens: int = 512, temperature: float = 0.7) -> str:
        """Genera texto a partir de un prompt"""
        messages = [{"role": "user", "content": prompt}]

        inputs = self.tokenizer.apply_chat_template(
            messages,
            return_tensors="pt",
            add_generation_prompt=True
        ).to(self.model.device)

        outputs = self.model.generate(
            inputs,
            max_new_tokens=max_tokens,
            temperature=temperature,
            do_sample=True,
            pad_token_id=self.tokenizer.eos_token_id
        )
        
        response = self.tokenizer.decode(outputs[0][inputs.shape[1]:], skip_special_tokens=True)
        return response.strip()

## 4. Configuración de Modelos Frontera (APIs)

In [ ]:
class OpenAIGenerator:
    """Generador usando OpenAI (GPT-4, GPT-4o, GPT-3.5)"""

    def __init__(self, model: str = "gpt-4o-mini", api_key: str = None):
        """
        Modelos disponibles:
        - gpt-4o: El más potente y reciente
        - gpt-4o-mini: Rápido y económico
        - gpt-4-turbo: Versión anterior de GPT-4
        - gpt-3.5-turbo: Más económico
        """
        from openai import OpenAI
        self.client = OpenAI(api_key=api_key or OPENAI_API_KEY)
        self.model = model

    def generate(self, prompt: str, max_tokens: int = 512, temperature: float = 0.7) -> str:
        response = self.client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=max_tokens,
            temperature=temperature
        )
        return response.choices[0].message.content


class AnthropicGenerator:
    """Generador usando Claude (Anthropic)"""

    def __init__(self, model: str = "claude-3-5-sonnet-20241022", api_key: str = None):
        """
        Modelos disponibles:
        - claude-3-5-sonnet-20241022: El más reciente y potente
        - claude-3-5-haiku-20241022: Rápido y económico
        - claude-3-opus-20240229: Máxima calidad (más caro)
        """
        from anthropic import Anthropic
        self.client = Anthropic(api_key=api_key or ANTHROPIC_API_KEY)
        self.model = model

    def generate(self, prompt: str, max_tokens: int = 512, temperature: float = 0.7) -> str:
        response = self.client.messages.create(
            model=self.model,
            max_tokens=max_tokens,
            temperature=temperature,
            messages=[{"role": "user", "content": prompt}]
        )
        return response.content[0].text


class GeminiGenerator:
    """Generador usando Gemini (Google)"""

    def __init__(self, model: str = "gemini-2.0-flash-exp", api_key: str = None):
        """
        Modelos disponibles:
        - gemini-2.0-flash-exp: Experimental, muy rápido
        - gemini-1.5-pro: Potente y con contexto largo
        - gemini-1.5-flash: Rápido y económico
        """
        import google.generativeai as genai
        genai.configure(api_key=api_key or GOOGLE_API_KEY)
        self.model = genai.GenerativeModel(model)

    def generate(self, prompt: str, max_tokens: int = 512, temperature: float = 0.7) -> str:
        response = self.model.generate_content(
            prompt,
            generation_config={
                'temperature': temperature,
                'max_output_tokens': max_tokens,
            }
        )
        return response.text

## 5. Clase Principal para Generación de Datasets

In [ ]:
class DatasetGenerator:
    """Clase principal para generar datasets con múltiples modelos"""

    def __init__(self):
        self.generators = {}
        self.results = []

    def add_generator(self, name: str, generator):
        """Añade un generador a la lista"""
        self.generators[name] = generator
        print(f"Generador '{name}' añadido")

    def generate_dataset(
        self,
        prompts: List[str],
        generator_names: Optional[List[str]] = None,
        max_tokens: int = 512,
        temperature: float = 0.7,
        delay: float = 1.0
    ) -> pd.DataFrame:
        """
        Genera un dataset usando los prompts y generadores especificados

        Args:
            prompts: Lista de prompts para generar
            generator_names: Nombres de los generadores a usar (None = todos)
            max_tokens: Máximo de tokens por generación
            temperature: Temperatura para la generación (0-2)
            delay: Tiempo de espera entre llamadas (para evitar rate limits)
        """
        if generator_names is None:
            generator_names = list(self.generators.keys())

        results = []

        total_iterations = len(prompts) * len(generator_names)
        pbar = tqdm(total=total_iterations, desc="Generando dataset")

        for prompt_idx, prompt in enumerate(prompts):
            for gen_name in generator_names:
                if gen_name not in self.generators:
                    print(f"⚠️ Generador '{gen_name}' no encontrado, saltando...")
                    continue

                try:
                    generator = self.generators[gen_name]
                    response = generator.generate(
                        prompt=prompt,
                        max_tokens=max_tokens,
                        temperature=temperature
                    )

                    results.append({
                        'prompt_id': prompt_idx,
                        'prompt': prompt,
                        'model': gen_name,
                        'response': response,
                        'temperature': temperature,
                        'max_tokens': max_tokens
                    })

                    pbar.set_postfix({'último_modelo': gen_name})

                except Exception as e:
                    print(f"\n❌ Error con {gen_name}: {str(e)}")
                    results.append({
                        'prompt_id': prompt_idx,
                        'prompt': prompt,
                        'model': gen_name,
                        'response': f"ERROR: {str(e)}",
                        'temperature': temperature,
                        'max_tokens': max_tokens
                    })

                pbar.update(1)
                time.sleep(delay)  # Evitar rate limits

        pbar.close()
        self.results.extend(results)

        df = pd.DataFrame(results)
        print(f"\n✅ Dataset generado: {len(df)} filas")
        return df

    def save_results(self, filename: str, format: str = 'csv'):
        """Guarda los resultados en el formato especificado"""
        df = pd.DataFrame(self.results)

        if format == 'csv':
            df.to_csv(filename, index=False)
        elif format == 'json':
            df.to_json(filename, orient='records', indent=2, force_ascii=False)
        elif format == 'jsonl':
            df.to_json(filename, orient='records', lines=True, force_ascii=False)
        elif format == 'parquet':
            df.to_parquet(filename, index=False)
        else:
            raise ValueError(f"Formato no soportado: {format}")

        print(f"💾 Resultados guardados en: {filename}")
        return filename

## 6. Ejemplos de Uso

### Ejemplo 1: Usar solo modelos de API (sin GPU)

In [ ]:
# Inicializar generador de datasets
dataset_gen = DatasetGenerator()

# Añadir generadores de APIs (no requieren GPU)
if OPENAI_API_KEY:
    dataset_gen.add_generator("gpt-4o-mini", OpenAIGenerator("gpt-4o-mini"))

if ANTHROPIC_API_KEY:
    dataset_gen.add_generator("claude-sonnet", AnthropicGenerator("claude-3-5-sonnet-20241022"))

if GOOGLE_API_KEY:
    dataset_gen.add_generator("gemini-flash", GeminiGenerator("gemini-2.0-flash-exp"))

### Ejemplo 2: Usar modelos de Hugging Face (requiere GPU)

In [ ]:
# Esta celda requiere GPU. Asegúrate de tener GPU habilitada:
# Runtime > Change runtime type > T4 GPU

# Cargar un modelo pequeño de Hugging Face
try:
    hf_gen = HuggingFaceGenerator(
        model_name="google/gemma-2-2b-it",  # Modelo pequeño, funciona en Colab gratuito
        use_4bit=True  # Cuantización para ahorrar memoria
    )
    dataset_gen.add_generator("gemma-2b", hf_gen)
except Exception as e:
    print(f"No se pudo cargar el modelo HF: {e}")
    print("Continuando solo con APIs...")

### Definir Prompts para tu Dataset

In [ ]:
# Ejemplo 1: Dataset de preguntas-respuestas en español
prompts_qa = [
    "¿Cuál es la capital de España y cuántos habitantes tiene?",
    "Explica qué es la inteligencia artificial en términos simples.",
    "¿Cuáles son los beneficios de hacer ejercicio regularmente?",
    "Describe el proceso de fotosíntesis en las plantas.",
    "¿Qué es el cambio climático y cuáles son sus principales causas?"
]

# Ejemplo 2: Dataset para generación de código
prompts_code = [
    "Escribe una función en Python que calcule el factorial de un número.",
    "Crea una función JavaScript que valide si un email es válido.",
    "Genera código SQL para crear una tabla de usuarios con id, nombre, email y fecha_registro."
]

# Ejemplo 3: Dataset de creative writing
prompts_creative = [
    "Escribe un párrafo describiendo un atardecer en la playa.",
    "Crea el inicio de un cuento de ciencia ficción sobre robots.",
    "Escribe un poema corto sobre la amistad."
]

# Selecciona qué prompts usar
prompts_to_use = prompts_qa  # Cambia esto según necesites

### Generar el Dataset

In [ ]:
# Generar el dataset
df = dataset_gen.generate_dataset(
    prompts=prompts_to_use,
    max_tokens=512,
    temperature=0.7,
    delay=1.0  # Espera 1 segundo entre llamadas
)

# Mostrar los primeros resultados
display(df.head(10))

### Análisis de Resultados

In [ ]:
# Estadísticas del dataset
print(f"Total de generaciones: {len(df)}")
print(f"Total de prompts únicos: {df['prompt_id'].nunique()}")
print(f"Modelos usados: {df['model'].unique().tolist()}")
print("\nDistribución por modelo:")
print(df['model'].value_counts())

# Longitud promedio de las respuestas
df['response_length'] = df['response'].str.len()
print("\nLongitud promedio de respuestas por modelo:")
print(df.groupby('model')['response_length'].mean().round(0))

### Guardar Resultados

In [ ]:
# Guardar en diferentes formatos
dataset_gen.save_results('dataset_generated.csv', format='csv')
dataset_gen.save_results('dataset_generated.json', format='json')
dataset_gen.save_results('dataset_generated.jsonl', format='jsonl')

# Descargar archivos (en Colab)
from google.colab import files
files.download('dataset_generated.csv')
files.download('dataset_generated.json')

## Ejemplo Avanzado: Generación con Variaciones de Temperatura

In [ ]:
# Generar con diferentes temperaturas para comparar creatividad
temperatures = [0.3, 0.7, 1.0]
prompt_test = "Escribe un párrafo sobre el futuro de la IA."

results_temp = []

for temp in temperatures:
    for gen_name, generator in dataset_gen.generators.items():
        try:
            response = generator.generate(
                prompt=prompt_test,
                max_tokens=200,
                temperature=temp
            )
            results_temp.append({
                'model': gen_name,
                'temperature': temp,
                'response': response
            })
        except Exception as e:
            print(f"Error: {e}")
        time.sleep(1)

df_temp = pd.DataFrame(results_temp)
display(df_temp)

## 📚 Recursos Adicionales

### Modelos de Hugging Face Recomendados (por tamaño):

**Pequeños (2-3B) - Funcionan en Colab gratuito:**
- `google/gemma-2-2b-it`
- `meta-llama/Llama-3.2-3B-Instruct`
- `Qwen/Qwen2.5-3B-Instruct`

**Medianos (7-8B) - Requieren Colab Pro:**
- `meta-llama/Llama-3.1-8B-Instruct`
- `mistralai/Mistral-7B-Instruct-v0.3`

### APIs de Modelos Frontera:

**OpenAI:** https://platform.openai.com/
- GPT-4o, GPT-4o-mini, GPT-3.5-turbo

**Anthropic:** https://console.anthropic.com/
- Claude 3.5 Sonnet, Claude 3.5 Haiku

**Google:** https://ai.google.dev/
- Gemini 2.0 Flash, Gemini 1.5 Pro

### Tips:
- Usa **temperatura baja (0.1-0.4)** para tareas que requieren precisión
- Usa **temperatura alta (0.7-1.2)** para tareas creativas
- Para datasets grandes, considera usar batch APIs cuando estén disponibles
- Guarda frecuentemente tus resultados para evitar pérdida de datos